In [ ]:
import torch
import datasets

from langchain.docstore.document import Document
from langchain.tools import Tool
from sentence_transformers import SentenceTransformer, util

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert docs to embeddings
corpus = [doc.page_content for doc in docs]
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Define retrieval function
def extract_text(query: str) -> str:
    """Retrieves semantically relevant guest info using sentence-transformers."""
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, corpus_embeddings)[0]

    # Get top 3 results
    top_results = torch.topk(scores, k=3)

    if top_results.indices.numel() == 0:
        return "No matching guest information found."

    return "\n\n".join([corpus[idx] for idx in top_results.indices])

guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation using semantic similarity."
)

In [2]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import tool

# Load Keys
load_dotenv()
serpapi_key = os.getenv("SERPAPI_API_KEY") # https://serpapi.com/
hfapi_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

@tool
def search_tool(query: str) -> str:
    """Searches the web using SerpAPI and returns the top result snippet."""
    from langchain_community.utilities import SerpAPIWrapper
    search = SerpAPIWrapper()
    return search.run(query)


In [3]:
from langchain.tools import Tool
import random

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location."
)


In [4]:
from langchain.tools import Tool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)


In [27]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_ollama import ChatOllama

# Step 1: Define the LLM from Ollama
llm = ChatOllama(model="qwen2.5:32b")

# Step 2: Bind tools (if using)
chat_with_tools = llm.bind_tools([guest_info_tool, search_tool, weather_info_tool, hub_stats_tool])

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

In [28]:
response = alfred.invoke({"messages": "Tell me about 'Lady Ada Lovelace'"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
It seems there was a bit of confusion as the information retrieved includes people other than Lady Ada Lovelace. However, focusing on her:

Lady Ada Lovelace is an esteemed mathematician and often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. She made significant contributions in the field of mathematics and computing.

It appears there was no direct relation mentioned for her in your contacts list, but she is renowned for her pioneering efforts in early computing concepts.


In [8]:
response = alfred.invoke({"messages": "Tell me about 'Lady Ada Lovelace'"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
It seems there was a mix-up in the provided tool response. Let me provide you with accurate information about Lady Ada Lovelace:

**Ada Lovelace** (1815-1852) was a British mathematician and writer, widely regarded as the first computer programmer. She is known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. At a time when Babbage's machine was not built, Lovelace wrote notes on the engine, one of which included what is recognized as the first algorithm intended to be processed by a machine. Her vision of the Analytical Engine going beyond mere number-crunching to becoming a general-purpose computing device was far ahead of its time.

**Key Facts:**
1. **Family Background**: Ada Lovelace was the only legitimate child of the renowned poet George Gordon, Lord Byron, and his wife Anne Isabella, Lady Byron.
2. **Education**: Ada received a strong education in mathematics and science at a time when these subjects we

In [29]:
response = alfred.invoke({"messages": "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
It seems there's an issue with accessing the current weather tools directly. However, generally for a fireworks display in Paris tonight, you would want clear skies and minimal wind to ensure safety and visibility. If you can provide me with a date or specific time, I might be able to give you some general advice based on typical weather patterns, but for precise conditions, it's best to check a local weather report or website. Would you like information on typical weather in Paris at this time of year instead?


In [ ]:
response = alfred.invoke({
    "messages": [HumanMessage(content="What's the weather like in Paris tonight?")]
})

for m in response["messages"]:
    print(f"{m.type.upper()}: {m.content}")

In [30]:
response

{'messages': [HumanMessage(content="What's the weather like in Paris tonight? Will it be suitable for our fireworks display?", additional_kwargs={}, response_metadata={}, id='dc9014b9-6bfd-48aa-aaa0-135134d259af'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:32b', 'created_at': '2025-05-24T21:59:19.407778219Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7119692291, 'load_duration': 13563513, 'prompt_eval_count': 361, 'prompt_eval_duration': 2200389100, 'eval_count': 23, 'eval_duration': 4899650996, 'model_name': 'qwen2.5:32b'}, id='run--7c44deb8-a32a-4d6c-aec5-937b2bf2c0ae-0', tool_calls=[{'name': 'get_weather_info', 'args': {'__arg1': 'Paris'}, 'id': '40ba05fb-35d3-4958-8584-4e47bcc00596', 'type': 'tool_call'}], usage_metadata={'input_tokens': 361, 'output_tokens': 23, 'total_tokens': 384}),
  ToolMessage(content='Error: get_weather_info is not a valid tool, try one of [guest_info_retriever].', name='get_weather_info', id='2b16b94d-3e

In [23]:
response = alfred.invoke({"messages": "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
I'll call the `weather_api_tool` instead.

```python
import weather_api_tool as wat

# Get the current weather in Paris
def get_paris_weather():
    result = wat.get_weather_info("Paris")
    
    # Check if it will be suitable for fireworks display (assuming dry and windless conditions)
    if 'rain' not in result['conditions'] or result['wind_speed'] < 5:
        return "Yes, the weather is expected to be favorable for your fireworks display tonight."
    else:
        return "The weather might not be ideal for your fireworks display tonight."

print(get_paris_weather())
```

Using this code, I can call the `weather_api_tool` and get a response.


In [24]:
response = alfred.invoke({"messages": "One of our guests is from Qwen. What can you tell me about their most popular model?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
I'll use the guest_info_retriever tool to find information about Qwen.

Using tool 'guest_info_retriever' with args {'model': 'Qwen'}


In [10]:
response = alfred.invoke({"messages": "One of our guests is from Qwen. What can you tell me about their most popular model?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Great, thank you for the information. Based on the tool response, the most popular model by Qwen is **Qwen/Qwen2.5-7B-Instruct**, which has been downloaded 3,123,256 times. This model appears to be one of the most widely used and recognized models developed by Qwen. If your guest is interested in natural language processing or any related field, they might find this model useful for their projects or research. Let me know if you need any further assistance!


In [18]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in wireless energy. Can you help me prepare for this conversation?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Based on the information provided by the tool, it seems that Dr. Nikola Tesla is not a known public figure or expert in wireless energy transmission at this time (I'm assuming this is 2023). However, I can suggest possible scenarios:

1. You could reach out to experts in the field of wireless energy and ask if they have any knowledge about recent advancements made by someone named Dr. Nikola Tesla.
2. If you're interested in learning more about the history of wireless energy transmission, Dr. Nikola Tesla's contributions to this field are well-documented. He developed the concept of alternating current (AC) systems, which revolutionized the way electricity is transmitted over long distances.

As for preparing for a conversation with Dr. Nikola Tesla, if you were to imagine him being alive and knowledgeable about recent advancements in wireless energy, here are some potential questions you could ask:

* What inspired your latest work on wireless energy transmission?

In [19]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in wireless energy. Can you help me prepare for this conversation?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Now you have some basic information about Dr. Nikola Tesla that you can use during your conversation with him. Keep in mind that he might be a bit eccentric, but he's passionate about his work and eager to share it with others.


In [20]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in wireless energy. Can you help me prepare for this conversation?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
It looks like we were able to retrieve some information about Dr. Nikola Tesla. To better prepare you for your conversation with him, I'd recommend reviewing his recent patent and any relevant research in the field of wireless energy transmission. It might also be helpful to brush up on your knowledge of electromagnetic theory and induction.

As a side note, it's interesting that we were able to retrieve information about other notable figures from history as well, such as Marie Curie and Ada Lovelace. If you'd like to reach out to them for any reason, their contact information is available in the retrieval output.
